In [2]:
import sys
sys.path.append('./src')
from OperatorSequence import *
from SparsePauliDynamics import *
from qiskit.quantum_info import PauliList
import numpy as np

### Simple example with 5 qubits

The circuit is U = (A * B)^3.
Observable is ZZIII.
A is composed of 5 Pauli rotation gates e^(-i theta * P / 2), where P are
YYIYI, IYYII, IIYIY, IYYYI, YIYII.
B is composed of 5 single qubit rotations around Z (ZIIII, IZIII, ...).
We order the operations by their order of acting on the observable in the Heisenberg picture, not by the action on the state 
(i.e., the first operator is the first one applied to the observable).

Rotation angles are all equal to 1.7. In our code we do not store theta, but theta/2 (i.e. coefficients c are stored and Pauli rotation gates are e^(-i * c * P)).
So the coefficients are 0.85 and must be provided as a list (of size n_layers, which is in this case 3 steps x 2 layers per step = 6) of lists 
(each list is of the size of the number of Pauli operators in a given layer).
Here, each layer contains 5 Pauli rotation gates, so the coefficients are provided as a list of 6 lists of lengths equal to 5.

OperatorSequence is a class that stores the circuit composed of Pauli rotation gates (it stores the lists of coefficients and Pauli operators).
Pauli operators are provided as a list of Qiskit's PauliList objects. PauliList objects can have different numbers of Paulis in principle (here they all contain 5 Paulis).

Once we have set up the observable and the OperatorSequence, we can initialize SPD Simulation (from SparsePauliDynamics module).
Simulation takes as input the observable, operator sequence, number of layers in the operator sequence (in our case 6), and a threshold (set to zero for exact simulation).
Once we have an instance sim of Simulation, we can run it with sim.run(), which will return the expectation value
<0| U^dag O U |0>, where U is our circuit defined by a sequence of Pauli rotation gates. It is returned as a complex number, although in most applications this will be a real number.


In [54]:
nq = 5
nsteps = 20
obs = PauliList(['IIZII'])
A_op = PauliList(['XXIII', 'IXXII', 'IIXXI', 'IIIXX', 'XIIIX'])
B_op = PauliList(['ZIIII', 'IZIII', 'IIZII', 'IIIZI', 'IIIIZ'])
ops = [A_op, B_op]  * nsteps
coeffs = [[0.3 for _ in range(nq)], [1.2 for _ in range(nq)] ] * nsteps
op_seq = OperatorSequence(ops, coeffs)

In [71]:
Lx = 5
Ly = 5

# ...
# 5, 6, 7, 8, 9
# 0, 1, 2, 3, 4

def ij_2_idx(i, j):
    return j*Lx + i

def idx_2_ij(idx):
    return idx%Lx, idx//5

def gen_ZZ_string(idx1, idx2):
    total_length = Lx * Ly
    str_list = ['I'] * total_length
    str_list[idx1] = 'Z'
    str_list[idx2] = 'Z'
    return ''.join(str_list)

def gen_Z_string(idx):
    total_length = Lx * Ly
    str_list = ['I'] * total_length
    str_list[idx] = 'Z'
    return ''.join(str_list)

def gen_X_string(idx):
    total_length = Lx * Ly
    str_list = ['I'] * total_length
    str_list[idx] = 'X'
    return ''.join(str_list)

# Add coupling
coupling_list = []
for j in range(Ly):
    for i in range(Lx-1):
        # print((i, j), (i+1, j), '=', ij_2_idx(i, j), '--', ij_2_idx(i+1, j))
        coupling_list.append((ij_2_idx(i, j), ij_2_idx(i+1, j)))
        
for i in range(Lx):
    for j in range(Ly-1):
        # print((i, j), (i, j+1), '=', ij_2_idx(i, j), '--', ij_2_idx(i, j+1))
        coupling_list.append((ij_2_idx(i, j), ij_2_idx(i, j+1)))

        
A_op_string = []
for i in range(len(coupling_list)):
    idx_x1 = coupling_list[i][0]
    idx_x2 = coupling_list[i][1]
    A_op_string.append(gen_ZZ_string(idx_x1, idx_x2))
    
B_op_string = []
for i in range(Lx*Ly):
    B_op_string.append(gen_X_string(i))

print(len(A_op_string))
print(len(B_op_string))
# print(gen_X_string(12))

# B_op_string = B_op_string + ['I'*Lx*Ly] * (len(A_op_string)-len(B_op_string))
# print(len(A_op_string))
# print(len(B_op_string))


40
25


In [72]:
nq = Lx * Ly
nsteps = 20
obs = PauliList([gen_Z_string((Lx*Ly//2))])
A_op = PauliList(A_op_string)
B_op = PauliList(B_op_string)
ops = [A_op, B_op] * nsteps
coeffs = [[0.3 for _ in range(len(A_op))], [1.2 for _ in range(len(B_op))] ] * nsteps
op_seq = OperatorSequence(ops, coeffs)

In [80]:
nsteps = 20
Z_list = []
for step in range(0, nsteps+1):
    print(step)
    sim = Simulation(obs, op_seq, depth=2*step, threshold=1e-4)
    Z_list.append(sim.run())
# assert(np.allclose(sim.run(), (0.005222428557558277+0j)))

import matplotlib.pyplot as plt
plt.plot(Z_list)
plt.show()

0
1
2
3
4
5


KeyboardInterrupt: 

In [79]:
print(Z_list)

[(1+0j), (-0.7373937155412454+0j), (0.6416025729350663+0j), (-0.4415314446492551+8.268550908866447e-17j), (0.1867650023062432-9.86588061242454e-18j), (-0.07066236648381906-2.0179388715070615e-17j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j]


In [4]:
sim = Simulation(obs, op_seq, depth=2*nsteps, threshold=0.0001)
assert(np.allclose(sim.run(), (0.005074072851170014-6.95027693594331e-18j)))

TypeError: expected dtype object, got 'numpy.dtype[uint64]'

### Clifford transformation

Before running the calculation, we can transform the angles to bring them into the range [-pi/4, pi/4] (or in the notation of the code, coefficients will be [-pi/8, pi/8]).
This is done at the OperatorSequence level using the reduce_clifford function. It will transform all the Pauli rotation gates inside and also store the Clifford operators
in clifford_ops. Then, the Clifford layer can be applied to the observable and now we are again left only with non-Clifford Pauli rotation gates that can be simulated with SPD.

As shown below, this procedure does not change the final results, but could affect the speed when using a non-zero threshold because it will minimize the number of insertions/deletions of Paulis.

In [5]:
q = 5
nsteps = 3
obs = PauliList(['ZZIII'])
A_op = PauliList(['YYIYI', 'IYYII', 'IIYIY', 'IYYYI', 'YIYII'])
B_op = PauliList(['ZIIII', 'IZIII', 'IIZII', 'IIIZI', 'IIIIZ'])
ops = [A_op, B_op]*nsteps 
coeffs = [[0.85 for _ in range(nq)] for i in range(2*nsteps)] 
op_seq = OperatorSequence(ops, coeffs)
op_seq.reduce_clifford()
obs = op_seq.clifford_ops.apply_to_PauliList(obs)

In [6]:
sim = Simulation(obs, op_seq, depth=2*nsteps, threshold=0.0)
assert(np.allclose(sim.run(), (0.0052224285575582854+1.528991154698683e-17j)))

TypeError: expected dtype object, got 'numpy.dtype[uint64]'

In [7]:
sim = Simulation(obs, op_seq, depth=2*nsteps, threshold=0.0001)
assert(np.allclose(sim.run(), (0.0050740728511700275-7.12841280550884e-17j)))

TypeError: expected dtype object, got 'numpy.dtype[uint64]'